In [1]:
%matplotlib inline
%load_ext autoreload
#%load_ext lab_black

In [2]:
from traffic.data.datasets import landing_zurich_2019 as t

/home/luis/.conda/envs/py37/lib/python3.7/site-packages/pyarrow/pandas_compat.py:294: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  elif isinstance(name, collections.Sequence):


In [3]:
# all flows
t_f = t.query("track == track").resample(30).unwrap("track").eval(max_workers=10)
t_f.to_parquet("data/lszh.parquet")
t_f.data.head()

,timestamp,altitude,callsign,geoaltitude,groundspeed,icao24,lastseen,latitude,longitude,onground,origin,track,vertical_rate,distance,flight_id,runway,initial_bearing,initial_flow,simple,track_unwrapped
0,2019-10-15 04:00:32+00:00,18400.0,EDW91,18975.0,369.423557,4b18ff,2019-10-15 04:30:24+00:00,46.991089,7.865387,False,SAEZ,52.601902,-1024.0,39.877133,EDW91_15202,14,44.294825,24-72,False,52.601902
1,2019-10-15 04:01:29+00:00,17475.0,EDW91,18000.0,364.005180,4b18ff,2019-10-15 04:30:24+00:00,47.049637,7.977448,False,SAEZ,52.815294,-1024.0,34.159029,EDW91_15202,14,44.294825,24-72,False,52.815294
2,2019-10-15 04:02:26+00:00,16525.0,EDW91,17025.0,351.489383,4b18ff,2019-10-15 04:30:24+00:00,47.113797,8.077486,False,SAEZ,43.962564,-1024.0,28.547228,EDW91_15202,14,44.294825,24-72,False,43.962564
3,2019-10-15 04:03:23+00:00,16000.0,EDW91,16500.0,323.210179,4b18ff,2019-10-15 04:30:24+00:00,47.178314,8.168472,False,SAEZ,43.871783,0.0,23.175342,EDW91_15202,14,44.294825,24-72,False,43.871783
4,2019-10-15 04:04:20+00:00,16000.0,EDW91,16500.0,298.006454,4b18ff,2019-10-15 04:30:24+00:00,47.243492,8.228760,False,SAEZ,346.203888,-64.0,18.633995,EDW91_15202,14,44.294825,24-72,False,-13.796112


In [9]:
stats = t.groupby("flight_id").agg({"runway": "max", "initial_flow": "max"})
stats = stats.reset_index().groupby(["runway", "initial_flow"]).count().unstack()
stats

flight_id                                 
initial_flow   162-216 24-72 240-276 312-354 90-132 N/A
runway                                                 
14                4437  2043    3430    1307   3166  58
16                   1     1       1       1      4   1
28                 899   565     677     323    667  31
34                 340   182     586     118    366   6
N/A                 76    24      90      29     22   6

In [5]:
# 1 flow
from traffic.core import Traffic
t_f = t.query("track == track and initial_flow == '162-216' and runway =='14'").resample(30).unwrap("track").eval(max_workers=10)
t_f.to_parquet("data/lszh1.parquet")
t_f.data.head()

,timestamp,altitude,callsign,geoaltitude,groundspeed,icao24,lastseen,latitude,longitude,onground,origin,track,vertical_rate,distance,flight_id,runway,initial_bearing,initial_flow,simple,track_unwrapped
0,2019-10-27 09:12:34+00:00,14375.0,DLH4YX,15175.0,290.247919,3c66e3,2019-10-27 09:29:44+00:00,48.128128,8.514954,False,EDDF,178.222426,-1216.0,39.852173,DLH4YX_2008,14,177.997495,162-216,True,178.222426
1,2019-10-27 09:13:01+00:00,13750.0,DLH4YX,14550.0,300.846877,3c66e3,2019-10-27 09:29:44+00:00,48.092465,8.530461,False,EDDF,157.526634,-1344.0,37.694566,DLH4YX_2008,14,177.997495,162-216,True,157.526634
2,2019-10-27 09:13:29+00:00,13125.0,DLH4YX,13900.0,298.999742,3c66e3,2019-10-27 09:29:44+00:00,48.056717,8.552582,False,EDDF,157.380135,-1216.0,35.540954,DLH4YX_2008,14,177.997495,162-216,True,157.380135
3,2019-10-27 09:13:56+00:00,12650.0,DLH4YX,13425.0,294.001447,3c66e3,2019-10-27 09:29:44+00:00,48.022842,8.573799,False,EDDF,157.185252,-1024.0,33.521756,DLH4YX_2008,14,177.997495,162-216,True,157.185252
4,2019-10-27 09:14:24+00:00,12225.0,DLH4YX,12975.0,283.769027,3c66e3,2019-10-27 09:29:44+00:00,47.988419,8.595291,False,EDDF,157.411198,-896.0,31.495638,DLH4YX_2008,14,177.997495,162-216,True,157.411198


In [3]:
# 2 flows
from traffic.core import Traffic
t_f = t.query("track == track and initial_flow in ['162-216', '90-132'] and runway in ['14', '28']").unwrap("track").resample(30).eval(max_workers=10)
t_f.to_parquet("data/lszh2.parquet")
t_f.data.head()

,timestamp,altitude,callsign,geoaltitude,groundspeed,icao24,lastseen,latitude,longitude,onground,origin,track,vertical_rate,distance,flight_id,runway,initial_bearing,initial_flow,simple,track_unwrapped
0,2019-11-24 16:05:52+00:00,13025.0,DLH6MH,13225.0,321.001280,3c675a,2019-11-24 16:21:56+00:00,48.128357,8.513306,False,EDDF,180.000000,-576.0,39.868276,DLH6MH_2095,14,177.906188,162-216,True,180.000000
1,2019-11-24 16:06:14+00:00,13000.0,DLH6MH,13200.0,317.999725,3c675a,2019-11-24 16:21:56+00:00,48.096143,8.513348,False,EDDF,180.000000,0.0,37.935464,DLH6MH_2095,14,177.906188,162-216,True,180.000000
2,2019-11-24 16:06:36+00:00,12950.0,DLH6MH,13150.0,319.999724,3c675a,2019-11-24 16:21:56+00:00,48.063510,8.513348,False,EDDF,180.000000,-256.0,35.977737,DLH6MH_2095,14,177.906188,162-216,True,180.000000
3,2019-11-24 16:06:57+00:00,12350.0,DLH6MH,12675.0,313.001327,3c675a,2019-11-24 16:21:56+00:00,48.033142,8.513306,False,EDDF,180.183053,-1728.0,34.156165,DLH6MH_2095,14,177.906188,162-216,True,180.183053
4,2019-11-24 16:07:19+00:00,11775.0,DLH6MH,12000.0,310.014248,3c675a,2019-11-24 16:21:56+00:00,48.002154,8.512894,False,EDDF,180.554458,-1664.0,32.298239,DLH6MH_2095,14,177.906188,162-216,True,180.554458


In [3]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from traffic.core import Traffic
from traffic.algorithms.clustering import prepare_features

list_features = ["track_unwrapped", "longitude", "latitude", "altitude"]

def to_npy(filename, reshape_type="ae"):
    t = Traffic.from_file(f"{filename}.parquet")
    nb_samples = len(t[0].data)
    nb_features = len(list_features)
    X = prepare_features(t, None, list_features)
    X = MinMaxScaler(feature_range=(-1, 1)).fit_transform(X)
    if reshape_type == "cae_keras":
        # CAE KERAS (batch_size, nb_samples, nb_features)
        X = X.reshape(-1, nb_samples, nb_features)
    elif reshape_type == "cae_torch":
        # CAE pytorch (batch_size, nb_features, nb_samples)
        X = X.reshape(-1, nb_features).T.reshape(-1, nb_features, nb_samples)
    np.save(f"{filename}_{reshape_type}.npy", X)

In [7]:
to_npy("data/lszh", "ae")

/home/luis/.conda/envs/py37/lib/python3.7/site-packages/pyarrow/pandas_compat.py:294: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  elif isinstance(name, collections.Sequence):


In [4]:
to_npy("data/lszh", reshape_type="cae_keras")

In [9]:
to_npy("data/lszh", reshape_type="cae_torch")